# `*`类型注释和检验

python3.5起就支持函数的类型注释(pep 484),它的结构如下:

In [1]:
def func(arg:int)->int:
    pass

***ps:类型注释只是注释,python解释器并不会处理它,要让它有类型检验的功能还要有其他工具配合.***

函数的参数类型保存在它的`__annotations__`属性上

In [2]:
func.__annotations__

{'arg': int, 'return': int}

## `*`自定义泛型注解


类型注释可以直接使用系统自带的类和自己定义的类,但对于泛型注解就力不从心了,对于这种需求,python内置了typing模块来帮助泛型注释

### 协程注释

In [3]:
async def spam(ignored: int) -> str:
    return 'spam'

async def foo() -> None:
    bar = await spam(42)  # type: str

### 类型别名

In [4]:
Url = str
def retry(url: Url, retry_count: int) -> None:
    pass

### 可调用类型

In [5]:
from typing import Callable

def feeder(get_next_item: Callable[[], str]) -> None:
    pass

def async_query(on_success: Callable[[int], None],
                on_error: Callable[[int, Exception], None]) -> None:
    pass

### 生成器类型

In [6]:
from typing import Generator
def echo_round() -> Generator[int, float, str]:
    res = yield
    while res:
        res = yield round(res)
    return 'OK'

In [7]:
from typing import Mapping, Set

def notify_by_email(employees: Set[int], overrides: Mapping[str, str]) -> None:
    pass

### 泛型

In [8]:
from typing import Sequence, TypeVar

T = TypeVar('T')      # Declare type variable

def first(l: Sequence[T]) -> T:   # Generic function
    return l[0]

### 受限泛型

In [9]:
from typing import TypeVar

AnyStr = TypeVar('AnyStr', str, bytes)#必须是str或者bytes

def concat(x: AnyStr, y: AnyStr) -> AnyStr:
    return x + y

###  Union类型

Union类型常用于可选类型


In [10]:
from typing import Union
def handle_employees(e: Union[int, Sequence[int]]) -> None:
    if isinstance(e, Employee):
        e = [e]

### Optional类型

Optional类型通常表示这个被注释的参数是可以为None的.

In [11]:
from typing import Optional
def option_demo(x:int,y: Optional[int]=None) -> int:
    if y:
        return x+y
    else:
        return x

### 用户自定义泛型

In [12]:
from typing import TypeVar, Generic
from typing import Iterable
class Logger:
    pass

T = TypeVar('T')

class LoggedVar(Generic[T]):
    def __init__(self, value: T, name: str, logger: Logger) -> None:
        self.name = name
        self.logger = logger
        self.value = value

    def set(self, new: T) -> None:
        self.log('Set ' + repr(self.value))
        self.value = new

    def get(self) -> T:
        self.log('Get ' + repr(self.value))
        return self.value

    def log(self, message: str) -> None:
        self.logger.info('{}: {}'.format(self.name,message))
        


def zero_all_vars(vars: Iterable[LoggedVar[int]]) -> None:
    for var in vars:
        var.set(0)

### any类型

any类型和ts中一样,代表任意类型都可以

### 方法重载

In [13]:
from typing import overload

class bytes:
    @overload
    def __getitem__(self, i: int) -> int: ...
    @overload
    def __getitem__(self, s: slice) -> bytes: ...

### 变量注解[3.6]

3.6版本起变量类型也可以注释了(pep 526),这看起来就像c语言一样,然而它依然还是注释

In [14]:
from typing import Optional,List
foo: Optional[int]
bar: List[str] = []

In [15]:
from typing import ClassVar

class C:
    x: int  # instance variable
    y: ClassVar[int]  # class variable
    z = None  # type: ClassVar[int]

    def foo(self) -> None:
        self.x = 0  # OK
        self.y = 0  # Error: Cannot assign to class variable "y" via instance

C.y = 0  # This is OK

模块,类中的的变量注解同样保存在`__annotations__`中

In [16]:
C.__annotations__

{'x': int, 'y': typing.ClassVar[int]}

In [17]:
c = C()

In [18]:
c.__annotations__

{'x': int, 'y': typing.ClassVar[int]}

In [19]:
__annotations__

{'bar': typing.List[str], 'foo': typing.Union[int, NoneType]}

## 静态类型检验

python解释器并不会做静态类型检验,我们可以利用[mypy](http://mypy-lang.org/)来实现

In [20]:
%%writefile src/C2/mypytest.py

from typing import Callable

def twice(i: int, next: Callable[[int], int]) -> int:
    return next(next(i))

def add(i: int) -> str:#写成返回str,这样就会报错!
    return i + 1

print(twice(3, add))   # 5

Overwriting src/C2/mypytest.py


In [21]:
!mypy src/C2/mypytest.py

src/C2/mypytest.py:8: error: Incompatible return value type (got "int", expected "str")
src/C2/mypytest.py:10: error: Argument 2 to "twice" has incompatible type "Callable[[int], str]"; expected "Callable[[int], int]"


## `*`运行时类型检测

标准库自带的typing只能用于静态检测,当我们需要运行时检测时可以借助[enforce](https://github.com/RussBaz/enforce)来实现.enforce使用装饰器语法,
它提供了装饰器 `@runtime_validation` 用于运行时进行类型检测.同时提供了工具`is_type_of_type`来对类型和申明类型进行比较.


需要注意的是`is_type_of_type`对通过cloudpickle的对象无效

In [22]:
!pip install enforce

Looking in indexes: http://pypi.douban.com/simple
ccxt 1.13.123 has requirement certifi>=2018.1.18, but you'll have certifi 2016.2.28 which is incompatible.
ccxt 1.13.123 has requirement requests>=2.18.4, but you'll have requests 2.14.2 which is incompatible.


In [23]:
import enforce

@enforce.runtime_validation 
class A:
    test:str
    def __init__(self,text:str)->None:
        self.text = text
        
@enforce.runtime_validation
def foo(text: str) -> A:
    return A(text)

In [24]:
a = foo("asd")

In [25]:
b = foo(123)

RuntimeTypeError: 
  The following runtime type errors were encountered:
       Argument 'text' was not of type <class 'str'>. Actual type was int.